In [1]:
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

from new_rpnet import Recognition_Module

/data2/home/shishirm/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
device = 'cuda:0'
torch.cuda.empty_cache()

In [3]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image_path = self.data_frame.iloc[idx, 0]
        left = self.data_frame.iloc[idx, 1:3]
        right = self.data_frame.iloc[idx, 3:5]
        label = [x for x in self.data_frame.iloc[idx, 5:]] # license plate chracters' indices

        image = Image.open(image_path).convert('RGB')
        shape = image.size # width x height, 

        box = [(left[0]+right[0])/(2*shape[0]), (left[1]+right[1])/(2*shape[1]), (right[0]-left[0])/shape[0], (right[1]-left[1])/shape[1]]
        # box = [cx, cy, w, h]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(box, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

In [4]:
transform = transforms.Compose([
    transforms.Resize((480, 480)),
    transforms.ToTensor(),
])

In [5]:
convert = torch.tensor([[1, 0, 1, 0], [0, 1, 0, 1], [-0.5, 0, 0.5, 0], [0, -0.5, 0, 0.5]]).to(device)
def compute_iou(pred, box):
    boxn = box.mm(convert).clamp(min=0, max=1)
    predn = pred.mm(convert).clamp(min=0, max=1)

    X_min = torch.max(boxn[:, 0], predn[:, 0])
    Y_min = torch.max(boxn[:, 1], predn[:, 1])
    X_max = torch.min(boxn[:, 2], predn[:, 2])
    Y_max = torch.min(boxn[:, 3], predn[:, 3])

    w = nn.functional.relu(X_max - X_min)
    h = nn.functional.relu(Y_max - Y_min)

    a_inter = w * h
    a1 = box[:, 2] * box[:, 3]
    a2 = pred[:, 2] * pred[:, 3]

    iou = a_inter / (a1 + a2 - a_inter)

    return torch.sum(iou).item()

In [6]:
def evaluate(model, dataset_path):
    dataset = CustomDataset(dataset_path, transform=transform)
    dataset_loader = DataLoader(dataset, batch_size=100, shuffle=False, num_workers=25, prefetch_factor=10)
    ch_correct = 0
    li_correct = 0
    total = 0
    iou = 0
    model.eval()
    with torch.no_grad():
        with tqdm(total = len(dataset_loader.dataset)) as pbar:
            for image, box, label in dataset_loader:
                image = image.to(device)
                box = box.to(device)
                label = label.to(device)

                pred_box, pred_label = model(image)

                predictions = [torch.argmax(curr, dim=1) for curr in pred_label]
                pred_license = torch.stack(predictions, dim=1).to(device)
                ch_equal = (label == pred_license)
                ch_correct += torch.sum(ch_equal).item()
                li_equal = torch.all(ch_equal, dim=1)
                li_correct += torch.sum(li_equal).item()
                iou += compute_iou(pred_box, box)
                total += image.shape[0]

                pbar.update(image.shape[0])
                
    print(f"Character Accuracy : {100 * ch_correct / (7 * total)} \tLicense Accuracy : {100 * li_correct / total} \tmIoU : {iou / total}")

In [7]:
model = Recognition_Module(device=device, path='./saved/detection_module.pth')
model.to(device)
model.load_state_dict(torch.load('./saved/prediction_module_30.pth', map_location=device))

<All keys matched successfully>

In [8]:
print("Results for CCPD Blur :")
evaluate(model, '../datasets/ccpd_blur.csv')

# e10 Character Accuracy : 75.59694199352634 	License Accuracy : 26.563485517442142 	mIoU : 0.6991262532849076
# e30 Character Accuracy : 77.32486813560027 	License Accuracy : 29.7511037795352 	mIoU : 0.7301453398418163


Results for CCPD Blur :


  0%|          | 0/20611 [00:00<?, ?it/s]

Character Accuracy : 77.32486813560027 	License Accuracy : 29.7511037795352 	mIoU : 0.7301453398418163


In [9]:
print("Results for CCPD Challenge :")
evaluate(model, '../datasets/ccpd_challenge.csv')

# e10 Character Accuracy : 79.40237871442 	License Accuracy : 33.62398256104634 	mIoU : 0.7203702927732402
# e30 Character Accuracy : 80.50202702123588 	License Accuracy : 36.20382777033378 	mIoU : 0.7488508395756935


Results for CCPD Challenge :


  0%|          | 0/50003 [00:00<?, ?it/s]

Character Accuracy : 80.50202702123588 	License Accuracy : 36.20382777033378 	mIoU : 0.7488508395756935


In [10]:
print("Results for CCPD DB :")
evaluate(model, '../datasets/ccpd_db.csv')

# e10 Character Accuracy : 76.92741526140658 	License Accuracy : 31.050138176075798 	mIoU : 0.6516729393261391
# e30 Character Accuracy : 78.7279341266708 	License Accuracy : 34.34662455586261 	mIoU : 0.6829328794889755


Results for CCPD DB :


  0%|          | 0/10132 [00:00<?, ?it/s]

Character Accuracy : 78.7279341266708 	License Accuracy : 34.34662455586261 	mIoU : 0.6829328794889755


In [11]:
print("Results for CCPD Rotate :")
evaluate(model, '../datasets/ccpd_rotate.csv')

# e10 Character Accuracy : 86.24007048357988 	License Accuracy : 46.951158858052324 	mIoU : 0.7579372485312706
# e30 Character Accuracy : 86.87811740631794 	License Accuracy : 49.28876952153586 	mIoU : 0.7846800199755505


Results for CCPD Rotate :


  0%|          | 0/10053 [00:00<?, ?it/s]

Character Accuracy : 86.87811740631794 	License Accuracy : 49.28876952153586 	mIoU : 0.7846800199755505


In [12]:
print("Results for CCPD Tilt :")
evaluate(model, '../datasets/ccpd_tilt.csv')

# e10 Character Accuracy : 82.43976701085518 	License Accuracy : 37.834259994704794 	mIoU : 0.7149307240498012
# e30 Character Accuracy : 83.20757214720678 	License Accuracy : 39.93910510987556 	mIoU : 0.7380510693288272


Results for CCPD Tilt :


  0%|          | 0/30216 [00:00<?, ?it/s]

Character Accuracy : 83.20757214720678 	License Accuracy : 39.93910510987556 	mIoU : 0.7380510693288272


In [13]:
print("Results for CCPD Weather :")
evaluate(model, '../datasets/ccpd_weather.csv')

# e10 Character Accuracy : 99.02847427599903 	License Accuracy : 94.95949594959495 	mIoU : 0.8289319364675726
# e30 Character Accuracy : 99.12419813409912 	License Accuracy : 95.42954295429543 	mIoU : 0.85375067658133


Results for CCPD Weather :


  0%|          | 0/9999 [00:00<?, ?it/s]

Character Accuracy : 99.12419813409912 	License Accuracy : 95.42954295429543 	mIoU : 0.85375067658133


In [14]:
print("Results for CCPD Base Test :")
evaluate(model, '../datasets/test.csv')

# e10 Character Accuracy : 99.67761367451362 	License Accuracy : 98.27830463841065 	mIoU : 0.8586276494399093
# e30 Character Accuracy : 99.70261409118723 	License Accuracy : 98.42330705511759 	mIoU : 0.8672984540993978


Results for CCPD Base Test :


  0%|          | 0/59999 [00:00<?, ?it/s]

Character Accuracy : 99.70261409118723 	License Accuracy : 98.42330705511759 	mIoU : 0.8672984540993978


In [15]:
print("Results for CCPD FN :")
evaluate(model, '../datasets/ccpd_fn.csv')

# e10 Character Accuracy : 72.77899283908728 	License Accuracy : 25.425668908284447 	mIoU : 0.6623283706805495
# e30 Character Accuracy : 73.69403620655588 	License Accuracy : 26.718176181618734 	mIoU : 0.681974960554935


Results for CCPD FN :


  0%|          | 0/20967 [00:00<?, ?it/s]

Character Accuracy : 73.69403620655588 	License Accuracy : 26.718176181618734 	mIoU : 0.681974960554935
